In [7]:
# 기본 패키지
import re
import time
import pickle
import pandas as pd
from tqdm import tqdm
from tqdm import trange
import warnings
warnings.filterwarnings('ignore')

# 크롤링 패키지
import requests
from selenium import webdriver
from bs4 import BeautifulSoup as bs


In [8]:
# 0. 페이지 한번 맨아래로 내리기
def scroll_bottom():
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

# 1. 입력한 위치로 설정하기
def set_location(location):
    print(location+'으로 위치 설정 하는중...')
    driver.find_element_by_css_selector('#search > div > form > input').click()
    driver.find_element_by_css_selector('#button_search_address > button.btn-search-location-cancel.btn-search-location.btn.btn-default > span').click()
    driver.find_element_by_css_selector('#search > div > form > input').send_keys(location)
    driver.find_element_by_css_selector('#button_search_address > button.btn.btn-default.ico-pick').click()
    time.sleep(5)
    print(location+'으로 위치 설정 완료!')
    
# 2-1. 카테고리 페이지 Element Number Dictionary 정의
food_dict = { '프랜차이즈':4, '치킨':5, '피자&양식':6, '중국집':7,
              '한식':8, '일식&돈까스':9, '족발&보쌈':10,
              '야식':11, '분식':12, '카페&디저트':13 }

# 2-2. 카테고리 페이지로 넘어가기
def go_to_category(category):
    print(category+' 카테고리 페이지 로드중...')
    driver.find_element_by_xpath('//*[@id="category"]/ul/li[{}]/span'.format(food_dict.get(category))).click()
    time.sleep(5)
    print(category+' 카테고리 페이지 로드 완료!')    

# 3. 카테고리(음식점 리스트) 페이지 모두 펼치기
def stretch_list_page():
    '''
    restaurant_count = int(driver.find_element_by_css_selector('#restaurant_count').text)
    scroll_count = int((restaurant_count/20))
    print('모든 음식점 리스트 불러오기 시작...')
    for _ in trange(scroll_count):
        try:
            scroll_bottom()
            time.sleep(3)
        except Exception as e:
            pass
    scroll_bottom()
    time.sleep(3)
    print('모든 음식점 리스트 불러오기 완료!')
    '''
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") # 스크롤을 가장 아래로 내린다
    time.sleep(5)
    pre_height = driver.execute_script("return document.body.scrollHeight") # 현재 스크롤 위치 저장
    while True :
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")  # 스크롤을 가장 아래로 내린다
        time.sleep(5)
        cur_height = driver.execute_script("return document.body.scrollHeight")  # 현재 스크롤을 저장한다.
        if pre_height == cur_height :
            break
        pre_height = cur_height
    time.sleep(3)
    print('모든 음식점 리스트 불러오기 완료!')
    
# 4. 해당 카테고리 음식점 리스트 리턴
def get_restaurant_list():
    restaurant_list=[]
    '''
    restaurants = driver.find_elements_by_css_selector('#content > div > div > div > div.restaurant-list > div.col-sm-6.contract')
    print("Loaded Restaurant list")
    print(restaurants)
    for restaurant in restaurants:
        restaurant_list.append(restaurant.find_element_by_css_selector('div > table > tbody > tr > td:nth-child(2) > div > div.restaurant-name.ng-binding').text)
    print(restaurant_list)
    '''
    html = driver.page_source
    html_source = bs(html, 'html.parser')
    restaurant_name = html_source.find_all("div", class_ = "restaurant-name ng-binding") #업체명
    for restaurant in restaurant_name:
        restaurant_list.append(restaurant.string)
    
    return list(set(restaurant_list))

# 5. 검색창에 음식점 검색하기
def search_restaurant(restaurant_name):
    try:
        driver.find_element_by_xpath('//*[@id="category"]/ul/li[1]/a').click()
        driver.find_element_by_xpath('//*[@id="category"]/ul/li[15]/form/div/input').send_keys(restaurant_name)    
        driver.find_element_by_xpath('//*[@id="category_search_button"]').click()
    except Exception as e:
        print('search_restaurant 에러')
    time.sleep(5)

# 6. 검색한 음식점 클릭
def go_to_restaurant():
    try:
        driver.find_element_by_xpath('//*[@id="content"]/div/div[5]/div/div/div[1]/div').click()
    except Exception as e:
        print('go_to_restaurant 에러')
    time.sleep(5)

# 7. 해당 음식점의 리뷰 페이지로 넘어가기
def go_to_review():
    print('리뷰 페이지 로드중...')
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a').click()
    time.sleep(2)
    print('리뷰 페이지 로드 완료!')
    
# 8. 리뷰 더보기 클릭하기 
def click_more_review():
    driver.find_element_by_class_name('btn-more').click()
    time.sleep(3)
    
# 9. 리뷰 페이지 모두 펼치기
def stretch_review_page():
    review_count = int(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span').text)
    click_count = int((review_count/10))
    print('모든 리뷰 불러오기 시작...')
    for _ in trange(click_count):
        try:
            scroll_bottom()
            click_more_review()
        except Exception as e:
            pass
    scroll_bottom()
    print('모든 리뷰 불러오기 완료!')
        
# 10. 해당 음식점의 모든 리뷰 객체 리턴
def get_all_review_elements():
    reviews = driver.find_elements_by_css_selector('#review > li.list-group-item.star-point.ng-scope')
    return reviews
        
# 11. 페이지 뒤로 가기 (한 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감)
def go_back_page():
    print('페이지 돌아가기중...')
    driver.execute_script("window.history.go(-1)")
    time.sleep(5)
    print('페이지 돌아가기 완료!'+'\n')
    
# 12. 크롤링과 결과 데이터를 pickle 파일로 저장
def save_pickle(location, category, yogiyo_df):
    pickle.dump(yogiyo_df, open('./{}_{}_df.pkl'.format(location, category),'wb'))
    print('{} {} pikcle save complete!'.format(location, category))

In [9]:
# 13. 크롤링 메인 함수
def yogiyo_crawling(location, category):
    df = pd.DataFrame(columns=['Restaurant','UserID','Menu','Review',
                                   'Total','Taste','Quantity','Delivery','Date'])
    
    try:
        set_location(location) # 검색할 지역 설정
        print("검색 지역 설정 완료")
        go_to_category(category) # 해당 카테고리(음식점 리스트) 페이지로 넘어감
        print("음식점 페이지 완료")

        print('Start [ {} - {} ] Crawling...'.format(location, category))
        
        stretch_list_page() # 카테고리(음식점 리스트) 페이지 모두 펼치기
        restaurant_list = get_restaurant_list() # 해당 카테고리 음식점 리스트 받아오기
        
        for restaurant_name in restaurant_list:
            try:
                print('********** '+restaurant_name+' ( '+str(restaurant_list.index(restaurant_name)+1)
                      +'/'+str(len(restaurant_list))+' 번째) **********')
                '''
                a = int(input())
                if(a==0):
                    continue
                '''
                search_restaurant(restaurant_name) # 음식점을 순서대로 검색
                print("음식점 찾기 완료")
                go_to_restaurant() # 검색한 음식점 클릭             
                go_to_review() # 해당 음식점의 리뷰페이지로 넘어감
                stretch_review_page() # 해당 음식점의 모든 리뷰를 불러옴
                prev_len = len(df)
                for review in tqdm(get_all_review_elements()):  # 해당 음식점의 리뷰 수 만큼 데이터를 가져옴
                    try:
                        df.loc[len(df)] = { 
                            'Restaurant':driver.find_element_by_class_name('restaurant-name').text,
                            'UserID':review.find_element_by_css_selector('span.review-id.ng-binding').text,
                            'Menu':review.find_element_by_css_selector('div.order-items.default.ng-binding').text,
                            'Review':review.find_element_by_css_selector('p').text,
                            'Total':str(len(review.find_elements_by_css_selector('div > span.total > span.full.ng-scope'))),
                            'Taste':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(3)').text,
                            'Quantity':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(6)').text,
                            'Delivery':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(9)').text,
                            'Date':review.find_element_by_css_selector('div:nth-child(1) > span.review-time.ng-binding').text,
                        }
                    except Exception as e:
                        print('리뷰 페이지 에러')
                        print(e)
                        pass
                    
            except Exception as e:
                print('*** '+restaurant_name+' *** 음식점 페이지 에러')
                #go_back_page()
                print(e)
                pass
            print(df.loc[prev_len:])
            print('음식점 리스트 페이지로 돌아가는중...')
            go_back_page() # 해당 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감
            
    except Exception as e:
        print('음식점 리스트 페이지 에러')
        print(e)
        pass
        
    print('End of [ {} - {} ] Crawling!'.format(location, category))
    save_pickle(location, category, df) # 해당 음식점 리뷰 데이터를 pickle 파일로 저장함
    #myslack.send_slack('{} {} crawling finish!!!'.format(location, category))
    
    return df 

In [10]:
# 14. 송파구 모든 관할구역(동) 크롤링 실행 함수
def start_songpagu_crawling(category):

    songpagu = ['가락동','거여동','마천동','문정동','방이동','삼전동',
                '석촌동','송파동','오금동','오륜동','잠실동','장지동','풍납동']
    songpagu = ['서울특별시 마포구 망원동 482-9 망원우체국']
    
    for dong in songpagu:
        try:
            yogiyo = yogiyo_crawling('{}'.format(dong), category)
            print(dong+' - '+category+', shape: '+str(yogiyo.shape)+'\n')
        except Exception as e:
            print('***** '+dong+' 에러 발생 *****')
            print(e)
            pass

---

In [11]:
# Chrome webdriver - 요기요 메인 페이지 실행
driverPath = "/Applications/chromedriver" # Chrome Driver path 
driver = webdriver.Chrome(driverPath) 
url = 'https://www.yogiyo.co.kr/'
driver.get(url)

In [12]:
%time start_songpagu_crawling('족발&보쌈')

서울특별시 마포구 망원동 482-9 망원우체국으로 위치 설정 하는중...
서울특별시 마포구 망원동 482-9 망원우체국으로 위치 설정 완료!
검색 지역 설정 완료
족발&보쌈 카테고리 페이지 로드중...
족발&보쌈 카테고리 페이지 로드 완료!
음식점 페이지 완료
Start [ 서울특별시 마포구 망원동 482-9 망원우체국 - 족발&보쌈 ] Crawling...
모든 음식점 리스트 불러오기 완료!
********** 홍대미남보쌈족발-서교점 ( 1/91 번째) **********
음식점 찾기 완료
리뷰 페이지 로드중...
리뷰 페이지 로드 완료!
모든 리뷰 불러오기 시작...


100%|█████████████████████████████████████████| 357/357 [17:58<00:00,  3.02s/it]


모든 리뷰 불러오기 완료!


100%|███████████████████████████████████████| 3180/3180 [16:32<00:00,  3.20it/s]


        Restaurant UserID                                               Menu  \
0     홍대미남보쌈족발-서교점  sh**님  보쌈족발 반반세트 小（보쌈＋족발）/1(족발 변경(변경 안 함),보쌈 변경(변경 안 ...   
1     홍대미남보쌈족발-서교점  tt**님                         나혼자족발세트/1(서비스 선택(서비스 막국수))   
2     홍대미남보쌈족발-서교점  ma**님                                           직화족발 小/1   
3     홍대미남보쌈족발-서교점  hy**님  나혼자불족발세트/1(불족발 맛 선택(1단계（기본 매운맛）),서비스 선택(서비스 주먹밥))   
4     홍대미남보쌈족발-서교점  ch**님  족발반반 小/1(족발 선택(족발,불족발 3단계（눈물콧물 찔끔 단계）),서비스 선택(...   
...            ...    ...                                                ...   
3175  홍대미남보쌈족발-서교점     손님                            족발 大/1(콜라 사이즈 변경(변경안함))   
3176  홍대미남보쌈족발-서교점  ky**님                                                      
3177  홍대미남보쌈족발-서교점  20**님                            족발 大/1(콜라 사이즈 변경(변경안함))   
3178  홍대미남보쌈족발-서교점     손님  족발 大/1(콜라 사이즈 변경(콜라 1.25L로 변경)),보쌈 大/1(콜라 사이즈 ...   
3179  홍대미남보쌈족발-서교점     손님  보쌈족발 반반세트 小（보쌈＋족발）/1(족발 변경(변경 안 함),서비스 선택(비빔메밀...   

                                       

...

0it [00:00, ?it/s]


모든 리뷰 불러오기 완료!


0it [00:00, ?it/s]

Empty DataFrame
Columns: [Restaurant, UserID, Menu, Review, Total, Taste, Quantity, Delivery, Date]
Index: []
음식점 리스트 페이지로 돌아가는중...
페이지 돌아가기중...


페이지 돌아가기 완료!

********** 천하제일족발보쌈 ( 89/91 번째) **********
음식점 찾기 완료
go_to_restaurant 에러
리뷰 페이지 로드중...
*** 천하제일족발보쌈 *** 음식점 페이지 에러
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="content"]/div[2]/div[1]/ul/li[2]/a"}
  (Session info: chrome=101.0.4951.41)
Stacktrace:
0   chromedriver                        0x000000010a6002c9 chromedriver + 5120713
1   chromedriver                        0x000000010a58ee33 chromedriver + 4656691
2   chromedriver                        0x000000010a17e158 chromedriver + 393560
3   chromedriver                        0x000000010a1b2edd chromedriver + 610013
4   chromedriver                        0x000000010a1b3161 chromedriver + 610657
5   chromedriver                        0x000000010a1e5244 chromedriver + 815684
6   chromedriver                        0x000000010a1d04ad chromedriver + 730285
7   chromedriver                        0x000000010a1e2f41 chromedriver + 806721
8   chromedriver                        0x

In [13]:
df = pickle.load(open('./서울특별시 마포구 망원동 482-9 망원우체국_족발&보쌈_df.pkl','rb'))

In [14]:
df.tail()

,Restaurant,UserID,Menu,Review,Total,Taste,Quantity,Delivery,Date
23204,원할머니보쌈-상암배달점,in**님,"맛있게 매운 의성마늘떡맵쌈/1(보쌈 사이즈 선택(중（생육710g이상,수육500g）))",맛과 양 모두 만족스럽습니다! 맛있게 잘 먹겠습니다~ 배달도 빠르고 포장도 너무 깔...,5,5,5,5,2021년 2월 10일
23205,장미로타리크럽,jk**님,"김.두.삼（김치두부삼겹살）/1,문어 초무침과 소면/1",배달이 좀 걸린듯 하지만 통두부에 삼겹김치.문어소면 맛있게 잘먹어요.폭풍흡입으로 사...,4,5,5,4,2020년 12월 1일
23206,장미로타리크럽,ja**님,문어삼합/1(사이즈 선택(3人)),여러번 먹었는데 맛있어요 사진이 없네요ㅠㅠ 호다닥 먹어서,5,5,5,5,2020년 11월 24일
23207,장미로타리크럽,손님,"유자보쌈（보쌈＋보쌈김치＋부추무침）/1(사이즈 선택(반근（300g）)),문어 초무침과...",친구네 놀러가서 시켜봤는데 진짜 전메뉴가 다 맛있다고 하는 친구말이 맞는거 같네요!...,5,5,4,4,2019년 1월 26일
23208,장미로타리크럽,wd**님,통다리 문어버터구이와 땅콩/1,그거알아요? 여기 메뉴 다맛있다는거☆ 항상오는 서비스 홍합탕 돈주고 시켜야되는거 아...,5,5,5,5,2019년 1월 23일


In [15]:
a = 1234
save_pickle("마포","족발&보쌈",a)

마포 족발&보쌈 pikcle save complete!


In [16]:
b = pickle.load(open('./마포_족발&보쌈_df.pkl','rb'))

In [17]:
print(b)

1234


In [18]:
# Chrome webdriver 종료
driver.close() 
driver.quit() 

---